# Swarm Pattern (LangGraph)

This notebook demonstrates a **Swarm Pattern** using LangGraph. Multiple sub-agents operate in parallel, share a common state, and iteratively influence the global outcome without a single controlling supervisor.

Assumption: Sub-agent Python modules already exist and expose callable agent functions.


## 0. Setup


In [ ]:
from dotenv import load_dotenv
import operator
from typing import List, TypedDict, Annotated

from langchain_core.messages import BaseMessage, HumanMessage
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, END

# Import swarm sub-agents (assumed available)
import finance.stock_research_agent as stock_research
import finance.industry_research_agent as industry_research
import finance.risk_agent as risk_agent


In [ ]:
# CHANGE THIS TO POINT TO YOUR OWN FILE
load_dotenv('C:\\Users\\raj\\.jupyter\\.env')

models = [
    {"model":"gpt-4.1-mini", "model_provider":"openai"},
    {"model":"command-r",  "model_provider":"cohere"},
    {"model": "llama-3.3-70b-versatile", "model_provider": "groq"},
]


## 1. Swarm State Schema


In [ ]:
class SwarmState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    stock_notes: str
    industry_notes: str
    risk_notes: str
    iteration: int


## 2. Initialize Sub-Agents


In [ ]:
llm = init_chat_model(**models[0])

stock_agent = stock_research.build_agent(llm)
industry_agent = industry_research.build_agent(llm)
risk_agent = risk_agent.build_agent(llm)


## 3. Swarm Nodes
Each node reads the shared state and appends its own analysis.


In [ ]:
def stock_node(state: SwarmState):
    result = stock_agent.invoke(state['messages'])
    return {"messages": result, "stock_notes": result[-1].content}

def industry_node(state: SwarmState):
    result = industry_agent.invoke(state['messages'])
    return {"messages": result, "industry_notes": result[-1].content}

def risk_node(state: SwarmState):
    result = risk_agent.invoke(state['messages'])
    return {"messages": result, "risk_notes": result[-1].content}


## 4. Aggregation Node
Lightweight synthesis step. In a swarm, this is not a controller — just a shared convergence point.


In [ ]:
def aggregate_node(state: SwarmState):
    summary_prompt = f"""
    You are a swarm aggregator. Combine the following perspectives into a coherent market view:
    Stock: {state.get('stock_notes','')}
    Industry: {state.get('industry_notes','')}
    Risk: {state.get('risk_notes','')}
    """
    response = llm.invoke([HumanMessage(content=summary_prompt)])
    return {"messages": [response], "iteration": state['iteration'] + 1}


## 5. Build Swarm Graph


In [ ]:
builder = StateGraph(SwarmState)

builder.add_node("stock", stock_node)
builder.add_node("industry", industry_node)
builder.add_node("risk", risk_node)
builder.add_node("aggregate", aggregate_node)

builder.set_entry_point("stock")
builder.add_edge("stock", "industry")
builder.add_edge("industry", "risk")
builder.add_edge("risk", "aggregate")
builder.add_edge("aggregate", END)

swarm_graph = builder.compile()


## 6. Run the Swarm


In [ ]:
initial_state = {
    "messages": [HumanMessage(content="Analyze NVDA as a long-term investment")],
    "stock_notes": "",
    "industry_notes": "",
    "risk_notes": "",
    "iteration": 0
}

result = swarm_graph.invoke(initial_state)
result
